In [1]:
import openreview
from deepdiff import DeepDiff
from pprint import pprint
from __future__ import print_function

guest_client = openreview.Client(baseurl="https://api.openreview.net")

def sort_by_tcdate(notes):
    if type(notes[0]) is dict:
        return list(sorted(notes, key=lambda x: x['tcdate']))
    else:
        return list(sorted(notes, key=lambda x: x.tcdate))

def print_metadata(note):
    if type(note) is dict:
        print(
            note['id'], note['forum'], 
            note['original'] if 'original' in note else None, 
            note['referent'] if 'referent' in note else None, 
            note['cdate'], note['tcdate'], note['tmdate'], note['invitation']
        )
    else:
        print(
            note.id, note.forum, note.original, note.referent,
            note.cdate, note.tcdate, note.tmdate, note.invitation
        )

In [2]:
VENUE = "ICLR.cc/2021/Conference"

In [3]:
# get all blind submissions
forum_notes = list(openreview.tools.iterget_notes(
    guest_client, invitation=f"{VENUE}/-/Blind_Submission", sort='tcdate',
))
for forum_note in forum_notes:
    assert forum_note.id == forum_note.forum and forum_note.id != forum_note.original
print('#BlindSubmission:', len(forum_notes))

#BlindSubmission: 2595


### Referent submission and comment notes

In [4]:
# get a blind submission
forum_note = forum_notes[0]
print('id forum original referent cdate tcdate tmdate invitation')
print_metadata(forum_note)
print(forum_note)

id forum original referent cdate tcdate tmdate invitation
trPMYEn1FCX trPMYEn1FCX mVImz0-gFe None 1601758676573 1601758676573 1614985710861 ICLR.cc/2021/Conference/-/Blind_Submission
{'cdate': 1601758676573,
 'content': {'_bibtex': '@misc{\n'
                        'bourached2021generative,\n'
                        'title={{\\{}GENERATIVE{\\}} '
                        '{\\{}MODEL{\\}}-{\\{}ENHANCED{\\}} {\\{}HUMAN{\\}} '
                        '{\\{}MOTION{\\}} {\\{}PREDICTION{\\}}},\n'
                        'author={Anthony Bourached and Ryan-Rhys Griffiths and '
                        'Robert Gray and Ashwani Jha and Parashkev Nachev},\n'
                        'year={2021},\n'
                        'url={https://openreview.net/forum?id=trPMYEn1FCX}\n'
                        '}',
             'abstract': 'The task of predicting human motion is complicated '
                         'by the natural heterogeneity and compositionality of '
                         'actions, 

In [5]:
# get the referent submission (forum) note and comment notes for this blind submission
this_forum_notes = sort_by_tcdate(list(openreview.tools.iterget_notes(guest_client, forum=forum_note.id)))
print('#NotesThisBlindSubmissionForum', len(this_forum_notes))

print('id forum original referent cdate tcdate tmdate invitation')
for note in this_forum_notes:
    print_metadata(note)
    if note.id == forum_note.id:
        print(DeepDiff(note, forum_note, ignore_order=True))
    else:
        comment_note = note
assert forum_note.id in [note.id for note in this_forum_notes]

#NotesThisBlindSubmissionForum 10
id forum original referent cdate tcdate tmdate invitation
trPMYEn1FCX trPMYEn1FCX mVImz0-gFe None 1601758676573 1601758676573 1614985710861 ICLR.cc/2021/Conference/-/Blind_Submission
{}
6KUeTkR9rOk trPMYEn1FCX None None 1603899183409 1603899183409 1605023935611 ICLR.cc/2021/Conference/Paper3828/-/Official_Review
k6EK7t1UUkU trPMYEn1FCX None None 1603924732695 1603924732695 1605023935540 ICLR.cc/2021/Conference/Paper3828/-/Official_Review
blQNVIlMJGk trPMYEn1FCX None None 1603974066584 1603974066584 1605023935468 ICLR.cc/2021/Conference/Paper3828/-/Official_Review
sK5JWeWRW_C trPMYEn1FCX None None 1603983910871 1603983910871 1606781753523 ICLR.cc/2021/Conference/Paper3828/-/Official_Review
obHLqSSHhs5 trPMYEn1FCX None None 1606154426349 1606154426349 1606154426349 ICLR.cc/2021/Conference/Paper3828/-/Official_Comment
XDz_nfKRCMr trPMYEn1FCX None None 1606154808935 1606154808935 1606154808935 ICLR.cc/2021/Conference/Paper3828/-/Official_Comment
humwfu3bgw

### Referent submission and its references

In [6]:
print("forum id\t", forum_note.id)
print("original id\t", forum_note.original)

forum id	 trPMYEn1FCX
original id	 mVImz0-gFe


In [7]:
# original submission and its revisions
submission_references_oT = sort_by_tcdate(guest_client.get_all_references(referent=forum_note.id, original=True))
print('#References:', len(submission_references_oT))

has_original = False
print('id forum original referent cdate tcdate tmdate invitation')
for ref in submission_references_oT:
    assert ref.original == forum_note.original
    print_metadata(ref)
    if ref.id == forum_note.original:
        print("above is the original submission note")
        has_original = True
    # TODO check that the original has the same contents as the forum
    # TODO only need one of this and the forum note
if not has_original:
    print("MISSING THE ORIGINAL NOTE")

#References: 2
id forum original referent cdate tcdate tmdate invitation
jmLYSMiHi trPMYEn1FCX mVImz0-gFe trPMYEn1FCX 1601758676573 1601758399224 1601758399224 ICLR.cc/2021/Conference/-/Blind_Submission
HF33Ewoz_Y trPMYEn1FCX mVImz0-gFe trPMYEn1FCX 1601758676573 1606153800359 1606153800359 ICLR.cc/2021/Conference/-/Blind_Submission
MISSING THE ORIGINAL NOTE


In [8]:
# this seems to return one of the above revisions, which is Code
guest_client.get_all_references(referent=forum_note.original, original=True)

[]

In [9]:
guest_client.get_all_references(referent=forum_note.original, original=False)

[]

In [10]:
print(forum_note)

{'cdate': 1601758676573,
 'content': {'_bibtex': '@misc{\n'
                        'bourached2021generative,\n'
                        'title={{\\{}GENERATIVE{\\}} '
                        '{\\{}MODEL{\\}}-{\\{}ENHANCED{\\}} {\\{}HUMAN{\\}} '
                        '{\\{}MOTION{\\}} {\\{}PREDICTION{\\}}},\n'
                        'author={Anthony Bourached and Ryan-Rhys Griffiths and '
                        'Robert Gray and Ashwani Jha and Parashkev Nachev},\n'
                        'year={2021},\n'
                        'url={https://openreview.net/forum?id=trPMYEn1FCX}\n'
                        '}',
             'abstract': 'The task of predicting human motion is complicated '
                         'by the natural heterogeneity and compositionality of '
                         'actions, necessitating robustness to distributional '
                         'shifts as far as out-of-distribution (OoD). Here we '
                         'formulate a new OoD benchmark ba

In [11]:
print(submission_references_oT[0]) # original

{'cdate': 1601758676573,
 'content': {'_bibtex': '@misc{\n'
                        'bourached2021generative,\n'
                        'title={{\\{}GENERATIVE{\\}} '
                        '{\\{}MODEL{\\}}-{\\{}ENHANCED{\\}} {\\{}HUMAN{\\}} '
                        '{\\{}MOTION{\\}} {\\{}PREDICTION{\\}}},\n'
                        'author={Anthony Bourached and Ryan-Rhys Griffiths and '
                        'Robert Gray and Ashwani Jha and Parashkev Nachev},\n'
                        'year={2021},\n'
                        'url={https://openreview.net/forum?id=trPMYEn1FCX}\n'
                        '}',
             'abstract': 'The task of predicting human motion is complicated '
                         'by the natural heterogeneity and compositionality of '
                         'actions, necessitating robustness to distributional '
                         'shifts as far as out-of-distribution (OoD). Here we '
                         'formulate a new OoD benchmark ba

In [12]:
print(submission_references_oT[1]) # revision

{'cdate': 1601758676573,
 'content': {'_bibtex': '@misc{\n'
                        'bourached2021generative,\n'
                        'title={{\\{}GENERATIVE{\\}} '
                        '{\\{}MODEL{\\}}-{\\{}ENHANCED{\\}} {\\{}HUMAN{\\}} '
                        '{\\{}MOTION{\\}} {\\{}PREDICTION{\\}}},\n'
                        'author={Anthony Bourached and Ryan-Rhys Griffiths and '
                        'Robert Gray and Ashwani Jha and Parashkev Nachev},\n'
                        'year={2021},\n'
                        'url={https://openreview.net/forum?id=trPMYEn1FCX}\n'
                        '}',
             'abstract': 'The task of predicting human motion is complicated '
                         'by the natural heterogeneity and compositionality of '
                         'actions, necessitating robustness to distributional '
                         'shifts as far as out-of-distribution (OoD). Here we '
                         'formulate a new OoD benchmark ba

### A referent comment note and its references

In [13]:
comment_references_oT = sort_by_tcdate(guest_client.get_all_references(referent=comment_note.id, original=True))
print('#References:', len(comment_references_oT))
print('id forum original referent cdate tcdate tmdate invitation')
for ref in comment_references_oT:
    print_metadata(ref)

#References: 1
id forum original referent cdate tcdate tmdate invitation
pw_Hqksml4 trPMYEn1FCX None YidG6-w3-Y 1610040431839 1610040431839 1610474031953 ICLR.cc/2021/Conference/Paper3828/-/Decision


In [14]:
comment_references_oF = sort_by_tcdate(guest_client.get_all_references(referent=comment_note.id, original=False))
print('#References:', len(comment_references_oF))
print('id forum original referent cdate tcdate tmdate invitation')
for ref in comment_references_oF:
    print_metadata(ref)

#References: 1
id forum original referent cdate tcdate tmdate invitation
pw_Hqksml4 trPMYEn1FCX None YidG6-w3-Y 1610040431839 1610040431839 1610474031953 ICLR.cc/2021/Conference/Paper3828/-/Decision


In [15]:
print(comment_note)

{'cdate': 1610040431839,
 'content': {'comment': 'This paper proposes a method for out-of-distribution '
                        'modeling and evaluation in  the human motion '
                        'prediction task. Paper was reviewed by four expert '
                        'reviewers who identified the following pros and '
                        'cons.\n'
                        '\n'
                        '> Pros:\n'
                        '- New benchmark for testing out of distribution '
                        'performance [R1]\n'
                        '- Compelling performance with respect to the '
                        'baselines [R1,R4]\n'
                        '- Paper is well written and easy to  follow  [R2]\n'
                        '- Generative model in the context  of  '
                        'out-of-distribution modeling of human motion is novel '
                        '[R1,R2,R4]\n'
                        '\n'
                        '> Cons:\n'
    

In [16]:
print(comment_references_oT[0])

{'cdate': 1610040431839,
 'content': {'comment': 'This paper proposes a method for out-of-distribution '
                        'modeling and evaluation in  the human motion '
                        'prediction task. Paper was reviewed by four expert '
                        'reviewers who identified the following pros and '
                        'cons.\n'
                        '\n'
                        '> Pros:\n'
                        '- New benchmark for testing out of distribution '
                        'performance [R1]\n'
                        '- Compelling performance with respect to the '
                        'baselines [R1,R4]\n'
                        '- Paper is well written and easy to  follow  [R2]\n'
                        '- Generative model in the context  of  '
                        'out-of-distribution modeling of human motion is novel '
                        '[R1,R2,R4]\n'
                        '\n'
                        '> Cons:\n'
    

In [17]:
print(comment_references_oF[0])

{'cdate': 1610040431839,
 'content': {'comment': 'This paper proposes a method for out-of-distribution '
                        'modeling and evaluation in  the human motion '
                        'prediction task. Paper was reviewed by four expert '
                        'reviewers who identified the following pros and '
                        'cons.\n'
                        '\n'
                        '> Pros:\n'
                        '- New benchmark for testing out of distribution '
                        'performance [R1]\n'
                        '- Compelling performance with respect to the '
                        'baselines [R1,R4]\n'
                        '- Paper is well written and easy to  follow  [R2]\n'
                        '- Generative model in the context  of  '
                        'out-of-distribution modeling of human motion is novel '
                        '[R1,R2,R4]\n'
                        '\n'
                        '> Cons:\n'
    

## Other exploration

### Get notes with direct or all replies

In [18]:
# get all blind submissions, each with its direct replies
forum_notes_dr = list(openreview.tools.iterget_notes(
    guest_client, invitation=f"{VENUE}/-/Blind_Submission", sort='tcdate',
    details='directReplies'
))
for forum_note in forum_notes:
    assert forum_note.id == forum_note.forum and forum_note.id != forum_note.original
print('#BlindSubmission:', len(forum_notes))

#BlindSubmission: 2595


In [19]:
# get all blind submissions, each with its replies
forum_notes_r = list(openreview.tools.iterget_notes(
    guest_client, invitation=f"{VENUE}/-/Blind_Submission", sort='tcdate',
    details='replies'
))
for forum_note in forum_notes:
    assert forum_note.id == forum_note.forum and forum_note.id != forum_note.original
print('#BlindSubmission:', len(forum_notes))

#BlindSubmission: 2595


In [20]:
# the first blind submission with direct replies
forum_note = forum_notes_dr[0]
print('#directReplies:', len(forum_note.details['directReplies']))
for reply in sort_by_tcdate(forum_note.details['directReplies']):
    print_metadata(reply)
print()
print_metadata(forum_note)
print(forum_note)

#directReplies: 5
6KUeTkR9rOk trPMYEn1FCX None None 1603899183409 1603899183409 1605023935611 ICLR.cc/2021/Conference/Paper3828/-/Official_Review
k6EK7t1UUkU trPMYEn1FCX None None 1603924732695 1603924732695 1605023935540 ICLR.cc/2021/Conference/Paper3828/-/Official_Review
blQNVIlMJGk trPMYEn1FCX None None 1603974066584 1603974066584 1605023935468 ICLR.cc/2021/Conference/Paper3828/-/Official_Review
sK5JWeWRW_C trPMYEn1FCX None None 1603983910871 1603983910871 1606781753523 ICLR.cc/2021/Conference/Paper3828/-/Official_Review
YidG6-w3-Y trPMYEn1FCX None None 1610040431839 1610040431839 1610474031953 ICLR.cc/2021/Conference/Paper3828/-/Decision

trPMYEn1FCX trPMYEn1FCX mVImz0-gFe None 1601758676573 1601758676573 1614985710861 ICLR.cc/2021/Conference/-/Blind_Submission
{'cdate': 1601758676573,
 'content': {'_bibtex': '@misc{\n'
                        'bourached2021generative,\n'
                        'title={{\\{}GENERATIVE{\\}} '
                        '{\\{}MODEL{\\}}-{\\{}ENHANCED{\

In [21]:
# the first blind submission with all replies
forum_note = forum_notes_r[0]
print('#replies:', len(forum_note.details['replies']))
for reply in sort_by_tcdate(forum_note.details['replies']):
    print_metadata(reply)
print()
print_metadata(forum_note)
print(forum_note)

#replies: 9
6KUeTkR9rOk trPMYEn1FCX None None 1603899183409 1603899183409 1605023935611 ICLR.cc/2021/Conference/Paper3828/-/Official_Review
k6EK7t1UUkU trPMYEn1FCX None None 1603924732695 1603924732695 1605023935540 ICLR.cc/2021/Conference/Paper3828/-/Official_Review
blQNVIlMJGk trPMYEn1FCX None None 1603974066584 1603974066584 1605023935468 ICLR.cc/2021/Conference/Paper3828/-/Official_Review
sK5JWeWRW_C trPMYEn1FCX None None 1603983910871 1603983910871 1606781753523 ICLR.cc/2021/Conference/Paper3828/-/Official_Review
obHLqSSHhs5 trPMYEn1FCX None None 1606154426349 1606154426349 1606154426349 ICLR.cc/2021/Conference/Paper3828/-/Official_Comment
XDz_nfKRCMr trPMYEn1FCX None None 1606154808935 1606154808935 1606154808935 ICLR.cc/2021/Conference/Paper3828/-/Official_Comment
humwfu3bgwS trPMYEn1FCX None None 1606155153639 1606155153639 1606155153639 ICLR.cc/2021/Conference/Paper3828/-/Official_Comment
UaNKPz8sASK trPMYEn1FCX None None 1606155835852 1606155835852 1606155835852 ICLR.cc/2021/

### Get references with other settings

In [22]:
# referent=forum_note.id, original=False
# -> for ICLR 2018 this returns the submission/forum note with little content
# -> here it returns some new note with little content
submission_references_oF = sort_by_tcdate(guest_client.get_all_references(referent=forum_note.id, original=False))
print('#References:', len(submission_references_oF))
assert len(submission_references_oF) == 1

print('id forum original referent cdate tcdate tmdate invitation')
for ref in submission_references_oF:
    print_metadata(ref)
    # assert ref.id == forum_note.id
    assert ref.original == forum_note.original

#References: 1
id forum original referent cdate tcdate tmdate invitation
3B-7TDoj17 trPMYEn1FCX mVImz0-gFe trPMYEn1FCX 1601758676573 1601758676573 1614985710861 ICLR.cc/2021/Conference/-/Blind_Submission


In [23]:
print(submission_references_oF[0])

{'cdate': 1601758676573,
 'content': {'_bibtex': '@misc{\n'
                        'bourached2021generative,\n'
                        'title={{\\{}GENERATIVE{\\}} '
                        '{\\{}MODEL{\\}}-{\\{}ENHANCED{\\}} {\\{}HUMAN{\\}} '
                        '{\\{}MOTION{\\}} {\\{}PREDICTION{\\}}},\n'
                        'author={Anthony Bourached and Ryan-Rhys Griffiths and '
                        'Robert Gray and Ashwani Jha and Parashkev Nachev},\n'
                        'year={2021},\n'
                        'url={https://openreview.net/forum?id=trPMYEn1FCX}\n'
                        '}',
             'reviewed_version_(pdf)': 'https://openreview.net/references/pdf?id=HF33Ewoz_Y'},
 'ddate': None,
 'details': None,
 'forum': 'trPMYEn1FCX',
 'id': '3B-7TDoj17',
 'invitation': 'ICLR.cc/2021/Conference/-/Blind_Submission',
 'mdate': None,
 'nonreaders': [],
 'number': 3828,
 'original': 'mVImz0-gFe',
 'readers': ['everyone'],
 'referent': 'trPMYEn1FCX',
 'replyto

### Seemingly useful but actually confusing

In [24]:
# get venue-specific event types
# the problem is that, unexpectedly, this doesn't return Official_Review
invitations = openreview.tools.iterget_invitations(
        guest_client, regex=f'{VENUE}.*'
)
event_types = set()
for invitation in invitations:
    event_types.add(invitation.id.split('/')[-1])
print(event_types)

{'Post_Decision_Revision', 'Withdrawn_Submission', 'Camera_Ready_Revision', 'Decision', 'Desk_Reject', 'Comment', 'Desk_Rejected_Submission', 'Form', 'Blind_Submission'}
